In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message="You are using `torch.load` with `weights_only=False`")
warnings.filterwarnings("ignore", category=UserWarning, message="The PyTorch API of nested tensors is in prototype stage and will change in the near future.")

%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0,"/root/gurusmart/MorganFP_prediction/reproduce_previous_works/Spectre")
            
import torch
torch.set_printoptions(precision=10)


import yaml
torch.set_float32_matmul_precision('high')
from pathlib import Path

from rdkit import Chem
from rdkit.Chem import Draw
# load model 
from datasets.dataset_utils import  fp_loader_configer

fp_loader_configer.select_version("Hash_Entropy")
fp_loader = fp_loader_configer.fp_loader

import numpy as np 
import random
seed=2
torch.cuda.manual_seed_all(seed) 
np.random.seed(seed)
random.seed(seed)



choosing Hash_Entropy_FP_loader


In [2]:
import pickle
with open(f'/root/gurusmart/MorganFP_prediction/inference_data/coconut_loutus_hyun_training/inference_metadata_latest_RDkit.pkl', 'rb') as file:
    smiles_and_names = pickle.load(file)
print(len(smiles_and_names))

from inference.inference_utils import choose_model
from inference.inference_utils import save_molecule_inference
from inference.inference_utils import retrieve_top_k_by_rankingset, compute_cos_sim, unpack_inputs_no_delimiter, build_input, inference_topK
from utils.get_NP_class import get_superclass_and_glycoside

526316


In [3]:
def get_smiles_from_dir(compound_dir):
    with open(Path(compound_dir) / "SMILES.txt", "r") as f:
        smiles = f.read().strip()
    return smiles

In [24]:
os.listdir("/root/gurusmart/MorganFP_prediction/reproduce_previous_works/Spectre/datasets/testing_compounds")

['new_compound_B1',
 'Aculeapyridone A',
 'Kavaratamide A',
 'new_compound_A',
 'Rhodomollein LVII',
 'Monchicamide I',
 'Wrightioside A',
 'Contignasterine A',
 'methyl_beta-D-glucopyranoside',
 'Glycyasymmetrica A',
 'Jejupeptin A',
 'Indanopyrrole A',
 'get_mw_weights_for_new_mols.ipynb',
 'new_compound_B2',
 'Fatuamide A']

In [25]:
compounds_names = [
    'new_compound_B1',
 'Aculeapyridone A',
 'Kavaratamide A',
 'new_compound_A',
 'Rhodomollein LVII',
 'Monchicamide I',
 'Wrightioside A',
 'Contignasterine A',
#  'methyl_beta-D-glucopyranoside',
 'Glycyasymmetrica A',
 'Jejupeptin A',
 'Indanopyrrole A',
 'new_compound_B2',
 'Fatuamide A']

# [
#  'Aculeapyridone A',
#  'Contignasterine A',
#  'Glycyasymmetrica A',
#  'Rhodomollein LVII',
#  'Monchicamide I',
#  'Wrightioside A',
#  'Jejupeptin A',
#  'Indanopyrrole A',
#  'Fatuamide A'
#  ]

In [20]:
def get_avg_cosine_in_annotation(model):
    all_cos = []
    for compound_dir in [f"/root/gurusmart/MorganFP_prediction/reproduce_previous_works/Spectre/datasets/testing_compounds/{name}" for name in compounds_names]:
        inputs, NMR_type_indicator = build_input(compound_dir, mode='None',
                            include_h_nmr=False, include_c_nmr=False, include_hsqc=True,
                            )
        inputs = inputs.unsqueeze(0).to(model.device)
        NMR_type_indicator = NMR_type_indicator.to(model.device)
    
        ground_smiles = get_smiles_from_dir(compound_dir)
        ground_FP = fp_loader.build_mfp_for_new_SMILES(ground_smiles).to(model.device)
        
        pred = model(inputs, NMR_type_indicator)
        pred = torch.sigmoid(pred) # sigmoid
        cosine = compute_cos_sim(pred, ground_FP)
        all_cos.append(cosine)
    all_cos = np.array(all_cos)
    avg_cos = np.mean(all_cos)
   
    return avg_cos


In [26]:
from collections import defaultdict

exp_dir = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/all_HSQC_jittering_search/"
jittering_dict = defaultdict(list)
for root, _, files in os.walk(exp_dir):
    
    for file in files:
        if file.endswith(".ckpt"):
            hparams, model = choose_model("", checkpoint_path=os.path.join(root, file))
            cos_score = get_avg_cosine_in_annotation(model)
            jitter = root.split("jittering_")[-1].split("-trial")[0]
            jittering_dict[jitter].append(cos_score)

for k, v in jittering_dict.items():
    print(f"jittering amount is {k}")
    print(np.mean(v), np.std(v))
    print("")


loading model from:  /root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/all_HSQC_jittering_search/only_hsqc_jittering_0.5-trial-1/checkpoints/epoch=88-step=38181.ckpt


Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args
Hash_Entropy_FP_loader is already setup
loading model from:  /root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/all_HSQC_jittering_search/only_hsqc_jittering_0.25-trial-2/checkpoints/epoch=50-step=21879.ckpt


Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args
Hash_Entropy_FP_loader is already setup
loading model from:  /root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/all_HSQC_jittering_search/only_hsqc_jittering_2-trial-1/checkpoints/epoch=71-step=30888.ckpt


...

HsqcRankedTransformer saving args
Hash_Entropy_FP_loader is already setup
loading model from:  /root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/all_HSQC_jittering_search/only_hsqc_jittering_0.5-trial-2/checkpoints/epoch=84-step=36465.ckpt


Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args
Hash_Entropy_FP_loader is already setup
loading model from:  /root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/all_HSQC_jittering_search/only_hsqc_jittering_1-trial-3/checkpoints/epoch=90-step=39039.ckpt


Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args
Hash_Entropy_FP_loader is already setup
jittering amount is 0.5
0.6786624995561746 0.009511074452518697

jittering amount is 0.25
0.6663055328222421 0.016661124762008567

jittering amount is 2
0.6937822882945722 0.006769678955399707

jittering amount is 0
0.6521557240914075 0.014063617548182236

jittering amount is 1
0.6971921049631559 0.014660908359677923



In [19]:
for k, v in jittering_dict.items():
    print(f"jittering amount is {k}")
    print(np.mean(v), np.std(v))
    print("")

jittering amount is epoch=90-step=39039
0.6580117841561636 0.0



'1'